In [2]:
from config import Config
from model.pytorch_models.cnn import CNN
from model.pytorch_models.cvt import ConvolutionalVisionTransformer, QuickGELU, LayerNorm
from model.pytorch_models.rvt import ResidualVisionTransformer
from __init__ import get_train_test_data
import torch
from torch import nn
from torchsummary import summary
import numpy as np
from functools import partial
from model.metrics import rmse, mape, mae, get_model_save_path
import tensorwatch as tw
import os

In [3]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
# is_cuda = torch.cuda.is_available()

# # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
# if is_cuda:
#     device = torch.device("cuda")
#     print("GPU is available")
# else:
#     device = torch.device("cpu")
#     print("GPU not available, CPU used")

# Set manual seed for reproducability
torch.manual_seed(50)

conf = Config("config_fig.yaml")
print(conf.observe_length)

12


In [4]:
# PREPARE DATA
data, arm_shape, train_xs, train_ys, train_arms, train_xp, train_xt, train_xe,\
    train_vehicle_type, train_engine_config, train_gen_weight,\
    test_xs, test_ys, test_arms, test_xp, test_xt, test_xe,\
    test_vehicle_type, test_engine_config, test_gen_weight = \
    get_train_test_data(conf, need_road_network_structure_matrix=True)  # \
print('************** Train - Predict **********')
print('train_xs:', train_xs.shape,  'test_xs:', test_xs.shape, 'train_xp:', train_xp.shape, 'test_xp:',
      test_xp.shape, 'test_xe:', test_xe.shape, 'train_ys:', train_ys.shape, 'test_ys:', test_ys.shape)
print(train_vehicle_type.shape, train_engine_config.shape, train_gen_weight.shape)


complete start
Start load_raw_link_speed_by_road_in_time ing..
speed items 363996
load from raw finish
function load_raw_link_speed_by_road cost 0.5527710914611816 s
preprocess finish
function load_raw_link_speed_by_road_in_time cost 0.5527710914611816 s
complete finish
STM  SHAPE(edges, timeslots)  13*28=  #######  (955, 364)
ARM  SHAPE(edges, A) ####### (955, 5)
T  SHAPE ####### 364
function completion_data cost 1.4211854934692383 s
****************** External Information ****************************
holiday shape (364, 1)
vec shape (day of week 7, weekend/weekday 1, hour of day 13 (364, 21)
External data shape (364, 22)
*********************************************************************
YS shape(,edges,predict_length) (364, 955, 1)
externel_data weather and holidays E.shape(,predict_length,) (364, 1, 22)
****************** Periodicity  Information ****************************
In-day periodicity XC.shape(nb_days,edges,timeslots_in_day) (28, 955, 12, 2)
Weekly periodicity XP.shape(4

In [5]:
# if conf.use_lookup:
#     train_xs = [train_xs, train_arms]
#     test_xs = [test_xs, test_arms]
if conf.use_vehicle_info:
    train_xs = [train_xs, train_vehicle_type, train_engine_config, train_gen_weight]
    test_xs = [test_xs, test_vehicle_type, test_engine_config, test_gen_weight]

if conf.use_externel:
    if conf.observe_p != 0:
        if isinstance(train_xs, list):
            train_xs += [train_xp]
            test_xs += [test_xp]
        else:
            train_xs = [train_xs, train_xp]
            test_xs = [test_xs, test_xp]

    if conf.observe_t != 0:
        if isinstance(train_xs, list):
            train_xs += [train_xt]
            test_xs += [test_xt]
        else:
            train_xs = [train_xs, train_xt]
            test_xs = [test_xs, test_xt]

    if conf.observe_p != 0 or conf.observe_t != 0:
        train_xs += [train_xe]
        test_xs += [test_xe]


In [11]:
print(train_xs[0].shape)
print(train_xs[1].shape)
print(train_xs[2].shape)
print(train_xs[3].shape)

(23, 955, 12, 2)
(23, 955, 12, 1)
(23, 955, 12, 1)
(23, 955, 12, 1)


In [21]:
print(train_xs[2])

[[[[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  ...

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[58]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]]


 [[[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  ...

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]]


 [[[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  [[25]
   [25]
   [25]
   ...
   [25]
   [25]
   [25]]

  ...

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[ 6]
   [ 6]
   [ 6]
   ...
   [ 6]
   [ 6]
   [ 6]]

  [[ 6

In [19]:
# np.concatenate((train_vehicle_type, train_engine_config), axis = 3)

In [12]:
model = None
if conf.model_name == 'CVT':
    model = ConvolutionalVisionTransformer(
        in_chans=1,
        num_classes=1,
        act_layer=QuickGELU,
        norm_layer=partial(LayerNorm, eps=1e-5),
        spec=conf.cvt_spec
    )
else:
    print("using RVT")
    model = ResidualVisionTransformer(
        in_chans=1,
        num_classes=1,
        act_layer=QuickGELU,
        norm_layer=partial(LayerNorm, eps=1e-5),
        spec=conf.cvt_spec
    )

using RVT


In [14]:
# We'll also set the model to the device that we defined earlier (default is CPU)
# model = model.to(device)

# Define hyperparameters
n_epochs = conf.epochs
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

summary(model, (1, 12, 2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 120, 12, 2]           1,200
            Conv2d-2           [-1, 120, 12, 2]         129,720
            Conv2d-3            [-1, 120, 3, 1]         518,520
         ResConv2d-4            [-1, 120, 3, 1]               0
         LayerNorm-5               [-1, 3, 120]             240
         ConvEmbed-6            [-1, 120, 3, 1]               0
           Dropout-7               [-1, 4, 120]               0
         LayerNorm-8               [-1, 4, 120]             240
            Conv2d-9            [-1, 120, 3, 1]           1,080
      BatchNorm2d-10            [-1, 120, 3, 1]             240
        Rearrange-11               [-1, 3, 120]               0
           Conv2d-12            [-1, 120, 2, 1]           1,080
      BatchNorm2d-13            [-1, 120, 2, 1]             240
        Rearrange-14               [-1,

C:\Users\binaya\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: can't multiply sequence by non-int of type 'list'